In [1]:
from datasets import load_dataset

In [2]:
dirs = """
cmg_ga_mu_corpusbeag
pmg_ga_co/RCPiarsachALL
nnc_ga_mu/CDNamedEntities01
nnc_ga_mu/CDAn_tOileanach_part3_newLTS
nnc_ga_mu/CDAn_tOileanach_part4_newLTS
nnc_ga_mu/CDNamedEntities02
nnc_ga_mu/CDAlphabet
nnc_ga_mu/CDAn_tOileanach_part2_newLTS
nnc_ga_mu/corpas_beag_newLTS
nnc_ga_mu/CDSaytime
nnc_ga_mu/CDAn_tOileanach_part1_newLTS
cmg_ga_mu/CMGOther
cmg_ga_mu/CMGCorpusBeag
anb_ga_ul/new_text02
anb_ga_ul/text01-004
anb_ga_ul/text01
anb_ga_ul/text01-001
anb_ga_ul/text02
anb_ga_ul/named-entities
nnc_en_ie
"""

In [3]:
dirl = [a for a in dirs.split("\n") if a]

In [18]:
script = "abairxml.py"

In [19]:
test = load_dataset(script, data_dir="/home/jim/tmp/cmg_ga_mu/CMGOther")

Using custom data configuration default-3a69a91aa0b88fa1
Reusing dataset utterance_xml_dataset (/home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-3a69a91aa0b88fa1/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b)


In [20]:
test

DatasetDict({
    train: Dataset({
        features: ['file_id', 'words', 'phonemes', 'audio', 'dialect', 'language', 'speaker_id', 'audio_set'],
        num_rows: 737
    })
})

In [21]:
test['train'][0]

{'file_id': 'CI0002CMGNamedEntities01_0202',
 'words': 'gaeilge na halban',
 'phonemes': ['g',
  'ee',
  'lj',
  'gj',
  '@',
  'n',
  '@',
  'h',
  'a',
  'l',
  '@',
  'b',
  '@',
  'n'],
 'audio': '/home/jim/tmp/cmg_ga_mu/CMGOther/wav16_trimmed/CI0002CMGNamedEntities01_0202.wav',
 'dialect': 'mu',
 'language': 'ga',
 'speaker_id': 'cmg',
 'audio_set': 'CMGOther'}

In [22]:
from pathlib import Path
def check_paths(batch):
    audio_path = Path(batch["audio"])
    return audio_path.is_file()

In [23]:
test.filter(check_paths)

Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-3a69a91aa0b88fa1/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-fd1748b2b209eed2.arrow


DatasetDict({
    train: Dataset({
        features: ['file_id', 'words', 'phonemes', 'audio', 'dialect', 'language', 'speaker_id', 'audio_set'],
        num_rows: 737
    })
})

In [24]:
sets = {}
for dir in dirl:
    sets[dir] = load_dataset(script, data_dir=f"/home/jim/tmp/{dir}")

Using custom data configuration default-187ee4831f664a6d
Reusing dataset utterance_xml_dataset (/home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-187ee4831f664a6d/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b)
Using custom data configuration default-aa8abe91265ea5f3
Reusing dataset utterance_xml_dataset (/home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-aa8abe91265ea5f3/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b)
Using custom data configuration default-7667bb9aedbb818e
Reusing dataset utterance_xml_dataset (/home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-7667bb9aedbb818e/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b)
Using custom data configuration default-7a61ec2b352637f4
Reusing dataset utterance_xml_dataset (/home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-7a61ec2b352637f4/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e07

In [25]:
from pathlib import Path
def check_paths(batch):
    audio_path = Path(batch["audio"])
    return audio_path.is_file()

for dir in dirl:
    sets[dir]["train"] = sets[dir]["train"].filter(check_paths)

Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-187ee4831f664a6d/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-96d34247636a824d.arrow
Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-aa8abe91265ea5f3/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-7018f164471d5b99.arrow
Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-7667bb9aedbb818e/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-4ae746a5bb8ab867.arrow
Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-7a61ec2b352637f4/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-65a1ed1f06c66730.arrow
Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-da3752ae2db1

In [26]:
!pip install torchaudio

Keyring is skipped due to an exception: Item does not exist!
Defaulting to user installation because normal site-packages is not writeable


In [27]:
import torchaudio

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["audio"])
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["words"]
    return batch

for dir in dirl:
    sets[dir]["train"] = sets[dir]["train"].map(speech_file_to_array_fn, remove_columns=sets[dir]["train"].column_names)

Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-187ee4831f664a6d/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-18df1d089931c98b.arrow
Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-aa8abe91265ea5f3/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-0eaad84bca78a073.arrow
Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-7667bb9aedbb818e/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-3c31f68123fd7809.arrow
Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-7a61ec2b352637f4/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-57b4fd0b447303a2.arrow
Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-da3752ae2db1

Something has gone wrong with `anb_ga_ul/text02`

Audio is in `wav_trimmed/`

In [28]:
import librosa
import numpy as np

def resample(batch):
    batch["duration"] = librosa.get_duration(y=batch["speech"], sr=batch["sampling_rate"])

    if batch["sampling_rate"] != 16_000:
        batch["speech"] = librosa.resample(np.asarray(batch["speech"]), batch["sampling_rate"], 16_000)
        batch["sampling_rate"] = 16_000

    return batch

for dir in dirl:
    sets[dir]["train"] = sets[dir]["train"].map(resample, num_proc=4)

Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-187ee4831f664a6d/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-da006be1caebfb5b.arrow
Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-187ee4831f664a6d/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-c2641d558f05e8f1.arrow
Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-187ee4831f664a6d/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-8d515b7aebd171c2.arrow
Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-187ee4831f664a6d/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-7b97b92998e9a71e.arrow
Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-aa8abe91265e

Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-394fefcb2267b139/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-bb045d0847c313cf.arrow
Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-394fefcb2267b139/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-fcb22388e1a65bcb.arrow
Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-e19a455adb1dd118/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-83fd8971f21f0642.arrow
Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-e19a455adb1dd118/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-4b82451aaabf6f8a.arrow
Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-e19a455adb1d

Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-336db2235fd49907/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-881af4492aebe81a.arrow
Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-336db2235fd49907/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-7aedf943d48feded.arrow
Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-336db2235fd49907/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-6f33f192208438a7.arrow
Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-336db2235fd49907/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-7ca896c968beb535.arrow
